<a href="https://colab.research.google.com/github/lcaspari/42_tensorflow_homework/blob/main/homework10/Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape, Layer

from tensorflow.keras.initializers import RandomUniform

import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from scipy.spatial import distance
from sklearn.manifold import TSNE

import re, math

# Load and preprocess the data

## Load the data

In [2]:
testData = tfds.load(name="tiny_shakespeare")["test"]


Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete0TGVN1/tiny_shakespeare-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete0TGVN1/tiny_shakespeare-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete0TGVN1/tiny_shakespeare-test.tfrecord


Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [3]:
for i in testData:
  print(i.keys())
  

dict_keys(['text'])


# 2. Word Embeddings

## 2.1 Preprocessing
- translate words to enitity id "castle" → 1450
  - convert to lowercase
  - remove punctuation and new line characters
- create two dictionaries containing the mappings
  - dict1: word → id
  - dict2: id → word
- create input-target pairs 
  - containing the input and its context (the target)
  - using a context window of 4 words
    - (word, context1)
    - (word, context2)
    - (word, context3)
    - (word, context4)

**outstanding**
→ disregard often occuring words with the given formular

In [4]:
import re

# remove punctuation
trainText = next(iter(testData))["text"].numpy()
print(trainText)

trainText = re.sub(r'\\n', ' ', str(trainText).lower())
trainText = re.sub(r"[^a-zA-Z ]", '', trainText)

print(trainText)

b"rance ta'en\nAs shall with either part's agreement stand?\n\nBAPTISTA:\nNot in my house, Lucentio; for, you know,\nPitchers have ears, and I have many servants:\nBesides, old Gremio is hearkening still;\nAnd happily we might be interrupted.\n\nTRANIO:\nThen at my lodging, an it like you:\nThere doth my father lie; and there, this night,\nWe'll pass the business privately and well.\nSend for your daughter by your servant here:\nMy boy shall fetch the scrivener presently.\nThe worst is this, that, at so slender warning,\nYou are like to have a thin and slender pittance.\n\nBAPTISTA:\nIt likes me well. Biondello, hie you home,\nAnd bid Bianca make her ready straight;\nAnd, if you will, tell what hath happened,\nLucentio's father is arrived in Padua,\nAnd how she's like to be Lucentio's wife.\n\nBIONDELLO:\nI pray the gods she may with all my heart!\n\nTRANIO:\nDally not with the gods, but get thee gone.\nSignior Baptista, shall I lead the way?\nWelcome! one mess is like to be your cheer

In [5]:
# assign ids to the words and create the dictionaries
vocab = {}

s = 0.001

calcProb = lambda x, z: (np.sqrt(z/s)+1)*(s/z)

# split the string
tokens = trainText.split(" ")
for index, token in enumerate(set(tokens)):
  # add the subsampling calucaltion
  # z = tokens.count(token)
  # prob = (np.sqrt(z/s)+1)*(s/z)
  vocab[token] = index

vocab_size = len(vocab)
inverse_vocab = {index: token for token, index in vocab.items()}

print(vocab_size)
print(vocab)
print(inverse_vocab)

2185
{'': 0, 'provision': 1, 'war': 2, 'dulness': 3, 'stoppd': 4, 'dukes': 5, 'now': 6, 'ild': 7, 'sightoutrunning': 8, 'counsellor': 9, 'parents': 10, 'butt': 11, 'joind': 12, 'tend': 13, 'mean': 14, 'spouse': 15, 'marketplace': 16, 'strangeness': 17, 'might': 18, 'hearkening': 19, 'conscience': 20, 'making': 21, 'wilt': 22, 'frown': 23, 'bride': 24, 'fen': 25, 'eye': 26, 'bound': 27, 'every': 28, 'coil': 29, 'safety': 30, 'poor': 31, 'consent': 32, 'marry': 33, 'ambition': 34, 'yellow': 35, 'boat': 36, 'lay': 37, 'place': 38, 'bourn': 39, 'sometime': 40, 'dozen': 41, 'princess': 42, 'speaks': 43, 'empty': 44, 'fresh': 45, 'spoken': 46, 'stronger': 47, 'beauty': 48, 'pray': 49, 'offices': 50, 'threats': 51, 'deceitful': 52, 'lusty': 53, 'coming': 54, 'seen': 55, 'labour': 56, 'ward': 57, 'lances': 58, 'men': 59, 'blasphemous': 60, 'point': 61, 'prepared': 62, 'dedicated': 63, 'pardon': 64, 'heaviness': 65, 'doublet': 66, 'stuffs': 67, 'brinepits': 68, 'hap': 69, 'anddo': 70, 'cradled'

In [6]:
# create input-context pairs
train_sequence = [vocab[word] for word in tokens]

window_size = 4
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      train_sequence, 
      vocabulary_size=vocab_size+1,
      window_size=window_size,
      negative_samples=0,
      categorical=True,
      shuffle=True)
print(len(positive_skip_grams))

75848


In [7]:
for target, context in positive_skip_grams[:8]:
  #if target == 2184:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(2050, 1045): (king, bank)
(2104, 1597): (ravens, my)
(561, 1794): (her, where)
(1715, 1987): (merry, you)
(1794, 1427): (where, they)
(923, 1168): (at, the)
(1471, 1168): (i, the)
(1606, 2109): (adrian, or)


In [8]:
batchSize = 32
bufferSize = len(positive_skip_grams)

asNum = np.array(positive_skip_grams)
test_dataset = tf.data.Dataset.from_tensor_slices((asNum[:,0], asNum[:,1]))

test_dataset = test_dataset.shuffle(bufferSize)
test_dataset = test_dataset.batch(batchSize)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)



In [9]:
for i in test_dataset:
  print(i)
  break

(<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 128, 2153,  397, 1877, 1371, 1264, 1278, 1182,  682, 1733,  232,
        826, 1168, 2176, 2049, 2008, 1562, 1492,  943, 1916, 2006, 1054,
        858, 1772, 2008, 1209, 1987, 2111, 1969,  192, 1072,  831])>, <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1209, 1980,  794,  557,  668,  835, 2008,  557,   45, 1780,  170,
       1309, 1431,  939, 1168,  586, 1227, 1987, 1877, 1256,  641, 1651,
       2172, 1378,  915,  557, 2184,  489, 1280,  839,  116, 1278])>)


## 2.2 Model
- implement a skip Gram model to create word embeddings
- use keras layer for the model
- overwrite the build function
  - use the tf.nn.nec_loss
  - only the outputs for the target and the sampled words can be computed this way
  - initalize the vocabulary in the __init__ function
  - get the embeddings in the call function
  - get the embeddings using tf.nn.embedding_lookup
  average the batch manually


In [11]:
class Word2Vec(Layer):
  def __init__(self, vocabSize, embeddingSize):
    super(Word2Vec, self).__init__()

    self.vocabSize = vocabSize
    self.embeddingSize = embeddingSize

  def build(self, InputShape):
    # goal is to optimize the embedding parameters 
    self.embedding = self.add_weight(
            shape=(self.vocabSize, self.embeddingSize),
            initializer = RandomUniform(minval=-1, maxval=1),
            trainable=True,
        )
    
    self.nce_weights = self.add_weight(
        shape=(self.vocabSize, self.embeddingSize),
        initializer="random_normal",
        trainable=True
    )
    self.nce_biases = self.add_weight(
        shape=(self.vocabSize,),
        initializer="random_normal",
        trainable=True
    )

  # train_inputs := source context
  # train_labels := target words  
  def call(self, train_inputs, train_labels, batchSize):
    # train_inputs should have the shape=[batch_size]
    # train_labels should have the shape=[batch_size, 1]
    train_labels = tf.expand_dims(train_labels, -1) 

    embed = tf.nn.embedding_lookup(self.embedding, train_inputs)

    # print(tf.shape(embed))
    # print(self.vocabSize)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights = self.nce_weights, 
            biases = self.nce_biases, 
            labels = train_labels, 
            inputs = embed, 
            num_sampled = 1, 
            num_classes = self.vocabSize, 
            num_true=1
    ))

    return loss



# 2.3 Training

In [21]:
def train_step(model, target, context, optimizer):
  with tf.GradientTape() as tape:
    loss = model(target, context, batchSize)
    gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss

def nearest_neighbours(target, k, embedding):
  wordEmbeddings = tf.nn.embedding_lookup(embedding, vocab[target])
  # print(wordEmbeddings.shape)
  # print(embedding.shape)

  mostSimilar = []
  for toCompate in wordEmbeddings:
    current = []
    for ii in embedding.numpy():
      current.append(distance.cosine(toCompate, ii))
    mostSimilar.append(np.argsort(np.array(current)[-(2+k):-2]))

  return [inverse_vocab[n] for n in mostSimilar], wordEmbeddings

def plotEmbedding(embeddings):
  tsne = TSNE(n_components=2, init='pca', random_state=0)
  X_tsne = tsne.fit_transform(embeddings)
  plot_embedding(X_tsne, "t-SNE embedding of the word embeddings")


def train_model(model, epochs, learning_rate, test_tokens):
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  for epoch in range(epochs):
    losses = []

    runningLoss = []
    for data in test_dataset:
      loss = train_step(model, data[0], data[1], optimizer)
      runningLoss.append(loss)

    losses.append(np.mean(runningLoss))
    print(f"Epoch: {epoch} loss: {loss}")

    embeddings = []
    for token in test_tokens:
      nearestWords, wordEmbeddings = nearest_neighbours(token, num_neighbours, model.embedding)
      print(wordEmbeddings.shape)
      print(f"  {token}: {', '.join(nearestWords)}")

    
    


In [22]:
epochs = 1
learning_rate = 0.01
embedding_size = 65
num_neighbours = 5 

test_tokens = ["gold", "authority", "wine", "death", "love", "faith", "day"]

model = Word2Vec(vocab_size, embedding_size)

train_model(model, epochs, learning_rate, test_tokens)

Epoch: 0 loss: 1.4814119338989258


TypeError: ignored

# 3 Text Generation

## 3.1 Preprocessing